In [ ]:
import os
import numpy as np

from sp_inference import processes, model, sampling, logging

In [ ]:
logSettings = {
    "output_directory":     "02_example_stationary",
    "verbose":              True
}

dataSettings = {
    "process_type":         "Dr31Di20Process",
    "drift_parameters":     [2, 3],
    "diffusion_parameters": [1, 2],
    "standard_deviation":   0.01,
    "rng_seed":             0,
    "num_points":           50,
    "domain_bounds":        [-1, 1]
}

modelSettings = {
    "params_to_infer":      "all",
    "model_type":           "mean_exit_time",
    "is_stationary":        True
}

priorSettings = {
    "mean_function":        ["-0.5*x", 1.5],
    "variance":             [0.01, 0.01],
    "correlation_length":   [0.01, 0.01],
    "robin_bc":             [True, True],
    "robin_bc_const":       [5, 5]
}

feSettings = {
    "num_mesh_points":      200,
    "boundary_locations":   [-1, 1],
    "boundary_values":      [0, 0],
    "element_degrees":      [1, 1]
}

solverSettings = {
    "rel_tolerance":         1e-6,
    "abs_tolerance":         1e-12,
    "max_iter":              50,
    "GN_iter":               5,
    "c_armijo":              1e-4,
    "max_backtracking_iter": 10
}

hessianSettings = {
    "num_eigvals":           10,
    "num_oversampling":      10
}

samplerSettings = {
    "algorithm":             'MALA',
    "use_gr_posterior":      True,
    "StepSize":              0.1
}

samplingRunSettings = {
    "NumSamples":            1000,
    "BurnIn":                100,
    "init_variance":         0.1,
    "init_seed":             0
}

In [ ]:
try:
    os.system('rm -r ' + logSettings["output_directory"])
except:
    pass

logger = logging.Logger(logSettings["verbose"],
                        logSettings["output_directory"])

In [ ]:
randGenerator = np.random.default_rng(dataSettings["rng_seed"])
randLocs = randGenerator.uniform(*dataSettings["domain_bounds"], dataSettings["num_points"])
dataSettings["domain_points"] = randLocs

processType = processes.get_process(dataSettings["process_type"])
process = processType(dataSettings["drift_parameters"],
                      dataSettings["diffusion_parameters"],
                      logger)

forwardNoisy, forwardExact = process.generate_data(modelSettings["model_type"],
                                                   modelSettings["is_stationary"],
                                                   dataSettings)
exactDrift = process.compute_drift(randLocs)
exactDiffusion = process.compute_squared_diffusion(randLocs)

misFitSettings = {
    "data_locations": randLocs,
    "data_values": forwardNoisy,
    "data_var": dataSettings["standard_deviation"]**2
}

In [ ]:
inferenceModel = model.SDEInferenceModel(modelSettings,
                                         priorSettings,
                                         feSettings,
                                         misFitSettings,
                                         logger=logger)
mapMeanData, mapVarianceData, mapForwardData, hessEigVals \
    = inferenceModel.compute_gr_posterior(solverSettings, hessianSettings)

In [ ]:
sampler = sampling.MCMCSampler(inferenceModel, samplerSettings, logger)
mcmcMeanData, mcmcVarianceData, mcmcForwardData = sampler.run(samplingRunSettings)
qoiTrace = sampler.evaluate_qoi()